In [1]:
import pickle
import dotenv

In [2]:
import tqdm
import time
import re

In [3]:
from mmw.parse import *

In [4]:
dotenv.load_dotenv()

True

In [5]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [6]:
all_parsed_essays = parse_essays()

In [7]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [8]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [9]:
model = "davinci:ft-mark-my-words-2023-05-04-11-12-35"
#model = "ada:ft-mark-my-words-2023-05-01-11-44-26"

In [10]:
import json

In [11]:
all_lines = []
with open('data/datasets/v2_short_format_with_question_explanation/finetune_set_prepared_valid.jsonl') as f:
    for line in f:
        all_lines.append(json.loads(line))

In [12]:
len(all_lines)

195

In [13]:
LIMIT_QUESTIONS = 200

In [14]:
all_answers = []
for i,line in tqdm.tqdm(enumerate(all_lines[:LIMIT_QUESTIONS])):
        
    prompt = line['prompt']

    response = openai.Completion.create(model=model, prompt=prompt, temperature=0, max_tokens=3)
    all_answers.append((0, 0, prompt, response, int(line['completion'])))
    
    time.sleep(.5)

195it [03:20,  1.03s/it]


In [15]:
len(all_answers)

195

In [16]:
all_answers[0]

(0,
 0,
 'Essay:\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time forwords and i\'m at school barely and my best friend walks towards me talking about\nhow her dad didn\'t buy her the new iphone 507989428 which is like a billion bucks while i\'m si

In [17]:
all_essays = set()
all_questions = set()
for cur in all_answers:
    cur_answer = cur[2]
    all_fount = list(cur_answer.split("\n\n###\n\n"))
    essay = all_fount[0]
    question = all_fount[1]
    
    all_essays.add(essay)
    all_questions.add(question)
    
all_essays = list(all_essays)
all_questions = list(all_questions)
for cur in range(len(all_answers)):
    cur_answer = all_answers[cur][2]
    all_fount = list(cur_answer.split("\n\n###\n\n"))
    essay = all_fount[0]
    question = all_fount[1]
    
    result = (all_essays.index(essay), all_questions.index(question), 
                        all_answers[cur][2], all_answers[cur][3], all_answers[cur][4])
    all_answers[cur] = result

In [18]:
all_answers[0]

(3,
 12,
 'Essay:\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time forwords and i\'m at school barely and my best friend walks towards me talking about\nhow her dad didn\'t buy her the new iphone 507989428 which is like a billion bucks while i\'m s

In [19]:
save_results('data/evaluations/run_5_ft-davinci_explanation', all_parsed_essays, all_questions, all_answers)

In [20]:
_,_,all_answers_loaded = load_results('data/evaluations/run_5_ft-davinci_explanation')

In [21]:
[(a[0],a[1],a[4],a[3]['choices'][0]['text']) for a in all_answers]

[(3, 12, 1, ' 0\n\n'),
 (3, 0, 1, ' 0\n\n'),
 (67, 11, 2, ' 3 3 3'),
 (67, 12, 2, ' 3 3 3'),
 (67, 4, 2, ' 2 3 3'),
 (67, 5, 3, ' 3 3 3'),
 (18, 12, 3, ' 3 3 3'),
 (18, 2, 3, ' 3 3 3'),
 (18, 5, 3, ' 3 3 3'),
 (18, 3, 3, ' 3\n\n'),
 (49, 12, 3, ' 3 3 3'),
 (49, 5, 3, ' 3 3 3'),
 (2, 11, 2, ' 2 2 2'),
 (2, 2, 1, ' 1 1 1'),
 (2, 8, 2, ' 2 2 2'),
 (2, 10, 1, ' 2 1 2'),
 (6, 2, 1, ' 0\n\n'),
 (14, 6, 3, ' 2 3 2'),
 (14, 4, 3, ' 2 1 3'),
 (14, 3, 0, ' 2 3 3'),
 (4, 11, 0, ' 2 2.'),
 (4, 0, 1, ' 2 2 2'),
 (4, 4, 1, ' 2 1 3'),
 (4, 5, 1, ' 2 3\n'),
 (4, 7, 2, ' 3 3.'),
 (54, 12, 3, ' 2.3'),
 (54, 6, 3, ' 2 3 3'),
 (66, 0, 2, ' 2 2 2'),
 (66, 7, 3, ' 3 3 3'),
 (59, 11, 2, ' 2 2 3'),
 (59, 0, 1, ' 2 2 2'),
 (59, 5, 2, ' 2 3\n'),
 (68, 11, 2, ' 1 1 2'),
 (68, 1, 1, ' 2 3 3'),
 (68, 0, 1, ' 2 2 2'),
 (68, 9, 1, ' 1 1 3'),
 (68, 8, 2, ' 2 2 2'),
 (12, 12, 1, ' 1 0 0'),
 (7, 9, 0, ' 1 1 1'),
 (7, 4, 1, ' 1 1 1'),
 (7, 3, 1, ' 1 1 1'),
 (27, 9, 2, ' 1 1 1'),
 (27, 2, 1, ' 0.0'),
 (27, 8, 2, ' 1 1 2'